In [1]:
import os
import cv2
import numpy as np

# Create the test directory to save frames
test_dir = "test"
os.makedirs(test_dir, exist_ok=True)

# Initialize the video capture from the webcam
video = cv2.VideoCapture(0)  # 0 indicates the default webcam

# Initialize frame counter and black frame counter
frame_count = 0
black_frame_count = 0
consecutive_black_frame_threshold = 30  # Adjust as needed

# Read frames from the webcam until a key is pressed or video is considered blacked out
while True:
    # Read a frame from the webcam
    ret, frame = video.read()

    # Convert the frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Threshold the grayscale frame to get a binary image
    _, thresholded = cv2.threshold(gray_frame, 30, 255, cv2.THRESH_BINARY)

    # Calculate the percentage of black pixels in the thresholded image
    black_pixel_ratio = np.mean(thresholded < 10)

    # Display the frame (optional)
    cv2.imshow("Webcam", frame)

    # Save the frame as an image in the test directory if the black pixel ratio is below a threshold
    if black_pixel_ratio < 0.9:
        frame_path = os.path.join(test_dir, f"frame_{frame_count}.jpg")
        cv2.imwrite(frame_path, frame)
        frame_count += 1
        black_frame_count = 0
    else:
        black_frame_count += 1

    # Check if consecutive black frames threshold is exceeded
    if black_frame_count >= consecutive_black_frame_threshold:
        print("Video is blacked out. Stopping capture.")
        break

    # Check for key press to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close the display window
video.release()
cv2.destroyAllWindows()

Video is blacked out. Stopping capture.


In [2]:
#Dataset
# https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset      
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report

In [3]:
# Set the seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Data directories
train_dir = "dataset/train"
valid_dir = "dataset/valid"

In [4]:
from PIL import Image

# Specify the path to the image
image_path = r'D:\Downloads\plant disease detection\dataset\New Plant Diseases Dataset(Augmented)\train\Apple___Apple_scab\0a5e9323-dbad-432d-ac58-d291718345d9___FREC_Scab 3417_90deg.JPG'  # Replace with the actual path to your image

# Open the image using PIL (Python Imaging Library)
image = Image.open(image_path)

# Get the size of the image
img_width, img_height = image.size

# Print the image size
print("Image size: {} x {}".format(img_width, img_height))

# Specify the number of classes (assumed for a classification task)
num_classes = 38

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Downloads\\plant disease detection\\dataset\\New Plant Diseases Dataset(Augmented)\\train\\Apple___Apple_scab\\0a5e9323-dbad-432d-ac58-d291718345d9___FREC_Scab 3417_90deg.JPG'

In [ ]:
# Data augmentation and preprocessing for the training set
train_datagen = ImageDataGenerator(
    
    # Rescale pixel values between 0 and 1
    rescale=1./255,  
    
    # Random rotation between -20 to +20 degrees
    rotation_range=20,  
    
    # Random horizontal shift within 20% of image width
    width_shift_range=0.2,  
    
    # Random vertical shift within 20% of image height
    height_shift_range=0.2,  
    
    # Random shearing transformation
    shear_range=0.2,  
    
    # Random zooming within 20%
    zoom_range=0.2, 
    
    # Randomly flip images horizontally
    horizontal_flip=True  
)

# Data preprocessing for the validation set to rescale pixel values between 0 and 1
valid_datagen = ImageDataGenerator(rescale=1./255)  

# Data preprocessing for the test set (if applicable) to rescale pixel values between 0 and 1
# test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Local directory path
train_dir = r'D:\Downloads\plant disease detection\dataset\New Plant Diseases Dataset(Augmented)\train'

# Create a data generator for training data
train_generator = train_datagen.flow_from_directory(
    
    # Directory containing the training data
    train_dir,  
    
    # Resize images to a specific width and height
    target_size=(img_width, img_height),  
    
    # Number of images in each batch
    batch_size=32,  
    
    # Type of labels (categorical for multi-class classification)
    class_mode='categorical'  
)

# Local directory path
valid_dir = r'D:\Downloads\plant disease detection\dataset\New Plant Diseases Dataset(Augmented)\valid'

# Create a data generator for validation data
valid_generator = valid_datagen.flow_from_directory(
    
    # Directory containing the validation data
    valid_dir,  
    
    # Resize images to a specific width and height
    target_size=(img_width, img_height),  
    
    # Number of images in each batch
    batch_size=32,  
    
    # Type of labels (categorical for multi-class classification)
    class_mode='categorical',  
    
    # Do not shuffle the data (preserve the order)
    shuffle=False  
)

In [ ]:
from tensorflow.keras.applications import DenseNet121

# Load the pre-trained DenseNet201 model
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Add custom classification layers on top of the pre-trained model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // valid_generator.batch_size
)

In [ ]:
# Save the trained model to a file
model.save('trained.h5')

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the pre-trained model
model = tf.keras.models.load_model('trained.h5')

# Evaluate the model on the validation generator
loss, accuracy = model.evaluate(valid_generator)

# Print the accuracy percentage
print('Accuracy: {:.2f}%'.format(accuracy * 100))

In [ ]:
import math

# Function to calculate Vapor Pressure Deficit (VPD)
def calculate_vpd(temperature, humidity):
    # Calculate saturation vapor pressure using the temperature
    saturation_vapor_pressure = 0.611 * math.exp((17.27 * temperature) / (temperature + 237.3))
    
    # Calculate actual vapor pressure using saturation vapor pressure and humidity
    actual_vapor_pressure = saturation_vapor_pressure * (humidity / 100)
    
    # Calculate Vapor Pressure Deficit (VPD) as the difference between saturation vapor pressure and actual vapor pressure
    vpd = saturation_vapor_pressure - actual_vapor_pressure
    
    return vpd

# Function to check if irrigation is needed based on VPD and humidity thresholds
def check_irrigation_needed(vpd_threshold, humidity_threshold):
    # Get temperature and humidity input from user
    temperature = float(input("Temperature (in degrees Celsius): "))
    humidity = float(input("Humidity (in percentage): "))
    
    # Calculate VPD using the provided temperature and humidity
    vpd = calculate_vpd(temperature, humidity)
    
    # Check if irrigation is needed based on VPD and humidity thresholds
    if vpd > vpd_threshold and humidity < humidity_threshold:
        print("Irrigation is needed for the leaves.")
    else:
        print("No irrigation is needed for the leaves.")

# Define the VPD and humidity thresholds (adjust these values based on plant requirements)
vpd_threshold = 1.5
humidity_threshold = 60

# Call the function to check if irrigation is needed
check_irrigation_needed(vpd_threshold, humidity_threshold)
